In [ ]:
%load_ext rich

In [ ]:
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import UnstructuredURLLoader
from rich import print
import re

In [ ]:
# clean blog post headers
pre_pattern = r"Skip to Main Content.*?AWS Machine Learning Blog"
post_pattern = r"\n\nComments.*?View Comments.*?All rights reserved\."

In [ ]:
url1 = "https://aws.amazon.com/blogs/machine-learning/use-aws-privatelink-to-set-up-private-access-to-amazon-bedrock"

loader = WebBaseLoader(url1)

# loader = UnstructuredURLLoader(
#     urls=[url1], mode="elements", strategy="fast",
# )
docs = loader.load()
print(len(docs))

# Clean extracted doc
page_content = docs[0].page_content
cleaned_page = re.sub(pre_pattern, "", page_content, flags=re.DOTALL)
cleaned_page = re.sub(post_pattern, "", cleaned_page, flags=re.DOTALL)
print(cleaned_page)

1

Use AWS PrivateLink to set up private access to Amazon Bedrock | AWS Machine Learning Blog





















































 



Use AWS PrivateLink to set up private access to Amazon Bedrock

        by 
       Ram Vittal, 
       Michael Daniels, and 
       Ray Khorsandi | on 
       30 OCT 2023 | in 
       Amazon Bedrock, Amazon Machine Learning, Best Practices, Generative AI, Intermediate (200) | 
       Permalink | 
        Comments | 
        Share


  
  
  
  





Amazon Bedrock is a fully managed service provided by AWS that offers developers access to foundation models (FMs) 
and the tools to customize them for specific applications. It allows developers to build and scale generative AI 
applications using FMs through an API, without managing infrastructure. You can choose from various FMs from Amazon
and leading AI startups such as AI21 Labs, Anthropic, Cohere, and Stability AI to find the model that’s best suited
for your use case. With the Amazon Bedrock serverless experience, you can quickly get started, easily experiment 
with FMs, privately customize them with your own data, and seamlessly integrate and deploy them into your 
applications using AWS tools and capabilities.
Customers are building innovative generative AI applications using Amazon Bedrock APIs using their own proprietary 
data. When accessing Amazon Bedrock APIs, customers are looking for mechanism to set up a data perimeter without 
exposing their data to internet so they can mitigate potential threat vectors from internet exposure. The Amazon 
Bedrock VPC endpoint powered by AWS PrivateLink allows you to establish a private connection between the VPC in 
your account and the Amazon Bedrock service account. It enables VPC instances to communicate with service resources
without the need for public IP addresses.
In this post, we demonstrate how to set up private access on your AWS account to access Amazon Bedrock APIs over 
VPC endpoints powered by PrivateLink to help you build generative AI applications securely with your own data.
Solution overview
You can use generative AI to develop a diverse range of applications, such as text summarization, content 
moderation, and other capabilities. When building such generative AI applications using FMs or base models, 
customers want to generate a response without going over the public internet or based on their proprietary data 
that may reside in their enterprise databases.
In the following diagram, we depict an architecture to set up your infrastructure to read your proprietary data 
residing in Amazon Relational Database Service (Amazon RDS) and augment the Amazon Bedrock API request with product
information when answering product-related queries from your generative AI application. Although we use Amazon RDS 
in this diagram for illustration purposes, you can test the private access of the Amazon Bedrock APIs end to end 
using the instructions provided in this post.

The workflow steps are as follows:

AWS Lambda running in your private VPC subnet receives the prompt request from the generative AI application.
Lambda makes a call to proprietary RDS database and augments the prompt query context (for example, adding product 
information) and invokes the Amazon Bedrock API with the augmented query request.
The API call is routed to the Amazon Bedrock VPC endpoint that is associated to the VPC endpoint policy with Allow 
permissions to Amazon Bedrock APIs.
The Amazon Bedrock service API endpoint receives the API request over PrivateLink without traversing the public 
internet.
You can change the Amazon Bedrock VPC endpoint policy to Deny permissions to validate that Amazon Bedrock APIs 
calls are denied.
You can also privately access Amazon Bedrock APIs over the VPC endpoint from your corporate network through an AWS 
Direct Connect gateway.

Prerequisites
Before you get started, make sure you have the following prerequisites:

An AWS account
An AWS Identity and Access Management 